In [ ]:
#Считывание из файла
df = pd.read_csv('dataset.csv')
df.info()

In [ ]:
#Обработка данных
df["head"] = df["head"].replace(-1, np.NaN) #Замена отсутствующих значений на NaN
df["head"] = df["head"] > 0 #Перевод в булевые значения
df = df.drop(columns = ["Trash"]) #Удаление лишнего
df.info()
print(df.duplicated().any()) #Проверка на дубликаты
df.dropna(how='all') #Проверка на пустые записи (thresh = n)

In [ ]:
#Рассчёт процента NaN данных
df_count = len(df)
print ("{:<45} {:<10} {:<10}".format('Характеристика', 'Кол-во NaN', 'Процент NaN')) 
for column in df.columns: 
    print("{:<50} {:<10} {:<10}".format(column, df[column].isnull().sum(), round(100 * df[column].isnull().sum() / df_count)))
df = df.drop(columns=["1", "2"]) # Удаление строчек с % NaN >= 35

In [ ]:
plt.plot((df.isnull().sum(axis = 1)).sort_values().values) #Вывод графика

df = df[df.isnull().sum(axis = 1) / len(df.columns) < 0.35]
df

In [ ]:
for column in df.columns:
    plt.figure()
    df.boxplot([column])
plt.show() #Вывод графика ящик с усами

In [ ]:
for column in df.columns: #Обработка выбросов и аномалий
    if column not in ["SL", "IHD"] and df.dtypes[column] != 'bool':
        values = df[column] 
        q1, q3 = values.quantile([0.25, 0.75])
        low  = q1 - (q3 - q1) * 5
        high = q3 + (q3 - q1) * 5
        condition = (values < low) | (values > high)
        values[condition] = np.nan
df

In [ ]:
df.describe() #Описание и оценка характеристик
df["SL"][df["SL"] < -1] = -1

In [ ]:
data = df.to_numpy() # Построение гистограмм
figure, axis = plt.subplots((df.dtypes=='float64').sum(), 1)
figure.set_size_inches(15, 250)
i = 0
for column in df.columns:
    if df.dtypes[column]=='float64':
        s = pd.Series(data[:, df.columns.get_loc(column)])
        axis[i].hist(s, bins = 10, log = True)
        axis[i].set_title(column)
        i += 1
plt.show() #Для визуальной оценки графиков

In [ ]:
i = 0 #Алгоритм рассчёта распределения данных
f = [None] * (df.dtypes=='float64').sum()
for column in df.columns:
    if df.dtypes[column]=='float64':
        f[i] = Fitter(df[column][~df[column].isnull()],
                   distributions=["expon", "uniform", "norm", "lognorm"],
                   bins = 15)
        time.sleep(0.2)
        f[i].fit()
        print(column)
        print(f[i].summary()['sumsquare_error'])
        i += 1  
#f[n].summary() для вывода отдельных примеров

In [ ]:
for column in df.columns: #Заполнение NaN значений
    if column not in ["RecordID"]:
        values = df[column] 
        E = values.mean()
        D = values.var()
        sigma = np.log(D / (E ** 2) + 1) ** 0.5
        mu = np.log(E) - (sigma ** 2) / 2
        condition = values.isna()
        new_values =  abs(np.random.lognormal(mu, sigma, len(df.index)))
        values[condition] = new_values[condition]
df
# Проверка, что NaN больше нет в датасете
df.isna().sum().sum()

In [ ]:
corr = df.corr() # Вывод матрицы корреляций
fig, ax = plt.subplots(figsize=(15, 10)) 
sns.heatmap(corr, ax=ax, xticklabels=True, yticklabels=True)

In [ ]:
# Пример-проверка, что NaN удалились в столбце, где изначально было 77% NaN
data = df.to_numpy()
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(15, 5)
s = pd.Series(data[:, df.columns.get_loc("1")])
print(len(s), len(df.index))
axis.hist(s, bins = 10, log = True)
axis.set_title("1")
plt.show()
df.to_csv('dataset2.csv', index = False)